# CWS / 子空间量子纠错码在非对称信道下的数值构造（Stiefel 流形优化 + 可选循环对称 k=0）

本 Notebook 来自你提供的脚本 **`stifiel_cyclic_print.py`** 的逐行拆解与“讲稿化”整理，并保持原始代码逻辑可直接运行。主要目标：

1. **构造一个 $n=L$ 量子比特系统上的 $K$ 维码空间**（一个子空间），用等距嵌入 $U\in\mathbb{C}^{2^L\times K}$ 表示；
2. 给定一个 **误差算符集合** $\mathcal{E}$（可切换：`orig/E1/E2/E3`，或按表格规格 `d='2'/'asym'` 生成），验证/逼近 **Knill–Laflamme (KL) 条件**：
   - **detect（检测级）**：对每个 $E\in\mathcal{E}\setminus\{I\}$，要求 $P E P \propto P$；
   - **correct（纠错级）**：对每个 $E_a,E_b\in\mathcal{E}$，要求 $P E_a^\dagger E_b P \propto P$；
3. 通过 **Stiefel 流形优化**（`numqi.manifold.Stiefel`）寻找 $U$，并在扫描 $\lambda^{*2}$（目标参数）时：
   - 实时记录优化历史；
   - 对每个目标点选出“**最优可行**”解并打印；
   - 画出曲线并保存 `png/csv` 摘要；
4. （可选）加入 **循环平移对称（cyclic symmetry，k=0）**：把码空间限制在循环不变子空间中，显著降维，加速搜索。

> 注：脚本内部把所有算符都转为稠密 `torch.complex128` 张量；当 $L$ 稍大（例如 $L\ge 7$）会非常吃内存/时间。请根据机器资源调整参数。

---

## 0. 依赖与运行环境

脚本依赖：

- `numpy`
- `torch`
- `scipy`（`scipy.sparse` 用于构造稀疏 Pauli 张量积矩阵）
- `matplotlib`
- `numqi`（提供 Stiefel 流形参数化与优化器）

如果你是 Conda 环境，典型安装方式（示例）：
```bash
pip install numpy scipy matplotlib
pip install torch --index-url https://download.pytorch.org/whl/cu121
pip install numqi
```

> Notebook 中我们尽量保持与原脚本一致；若你想把“稀疏矩阵”一路保留到损失计算处，可进一步做稀疏/块对角优化，但那会改动较大，不在本整理范围内。

In [ ]:
# ===== 0. 基础导入 & 线程设置 =====
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
import scipy.sparse as sp
from scipy.sparse import csr_matrix
from itertools import combinations
from matplotlib.ticker import LogLocator, ScalarFormatter

import numqi

# 线程：与原脚本保持一致（可按需改）
os.environ['MKL_NUM_THREADS'] = '12'
os.environ['OMP_NUM_THREADS'] = '12'
if torch.get_num_threads() != 12:
    torch.set_num_threads(12)

print("[setup] PyTorch threads =", torch.get_num_threads())

## 1. 统一配置区（你最常改的部分）

原脚本把配置分成两套：

- **新风格（按表格规格）**：`USE_TABLE_SPEC=True`  
  通过 `(n=L, K, d, r)` 生成误差集合：
  - `d='2'`：对称误差集 $\{I\}\cup\{X_i,Y_i,Z_i\}$
  - `d='asym', r=None`：混合非对称：$\{I\}\cup\{X_i,Y_i,Z_i\}\cup\{XX,ZZ,XZ,ZX\}$
  - `d='asym', r>=1`：$\{I\}\cup\{X_i,Y_i\}\cup Z_{\le r}$

- **旧风格（手选误差集模式）**：`ERROR_SET_MODE='orig'|'E1'|'E2'|'E3'`

此外还有：

- `KL_LEVEL='detect'|'correct'`
- `lambda2_list`：扫描的 $\lambda^{*2}$ 网格
- `optim_kwargs`：numqi 优化器参数（重复次数/阈值等）
- `constraint_threshold`：判断“可行解”的阈值（offdiag+diag 小于该阈值才算满足 KL 约束）

以及新增的循环对称选项：

- `USE_CYCLIC_SYM=False/True`（目前仅实现 `k=0`）

下面给出一个**可直接运行**的参数块，并提供一个更快的“烟雾测试（smoke test）”配置。

In [ ]:
# ===== 1) 表格规格 / 误差集规格 =====
USE_TABLE_SPEC = True
TABLE_N = 5          # n = L
TABLE_K = 3          # 码空间维 K
TABLE_D = 'asym'     # '2' or 'asym'
TABLE_R = 2          # r；表里“–”可用 None

# 旧风格（当 USE_TABLE_SPEC=False 时生效）
ERROR_SET_MODE = 'orig'   # 'orig'|'E1'|'E2'|'E3'

# ===== 2) 统一 L 与 K =====
L = TABLE_N if USE_TABLE_SPEC else 6
CODE_K = TABLE_K

# ===== 3) KL 级别 =====
KL_LEVEL = 'detect'         # 'detect' 或 'correct'
DROP_IDENTITY_IN_DETECT = True

# ===== 4) 扫描 λ*² 的网格 =====
lambda2_list = np.linspace(0.0, 1.5, 100)
# 确保关键点在网格中（可选）
for x_ins in (0.6, 1.0):
    if x_ins not in lambda2_list:
        lambda2_list = np.insert(lambda2_list, np.searchsorted(lambda2_list, x_ins), x_ins)

# ===== 5) 优化器超参 =====
optim_kwargs = dict(
    theta0='uniform',
    num_repeat=50,
    tol=1e-15,
    print_freq=0,
    early_stop_threshold=1e-14,
)
constraint_threshold = 1e-12

# ===== 6) 循环对称限制（可选）=====
USE_CYCLIC_SYM = False      # True = 限制到 cyclic k=0 子空间
CYCLIC_SECTOR = 'k0'

# ===== 7) 一键切换：快速 smoke test（建议第一次先跑这个）=====
SMOKE_TEST = False
if SMOKE_TEST:
    lambda2_list = np.array([0.0, 0.2, 0.6, 1.0])
    optim_kwargs = dict(theta0='uniform', num_repeat=5, tol=1e-12, print_freq=0, early_stop_threshold=1e-10)
    constraint_threshold = 1e-9

print(f"[config] L={L}, CODE_K={CODE_K}, KL={KL_LEVEL}, TABLE(d={TABLE_D}, r={TABLE_R}, K={TABLE_K}), cyclic={USE_CYCLIC_SYM}")

## 2. Pauli 张量积算符的构造（稀疏）

我们用标准 Pauli：

- $X=\begin{pmatrix}0&1\\1&0\end{pmatrix}$
- $Y=\begin{pmatrix}0&-i\\i&0\end{pmatrix}$
- $Z=\begin{pmatrix}1&0\\0&-1\end{pmatrix}$

对 $L$ 比特系统，单比特算符 $X_j$ 表示在第 $j$ 个比特上作用 $X$，其余为 $I$：
\[
X_j = I^{\otimes (j-1)} \otimes X \otimes I^{\otimes (L-j)}.
\]

这里用 `scipy.sparse` 构造稀疏矩阵，避免在“生成误差集合”阶段就爆内存；但注意随后优化时会把所有算符转为稠密 `torch.complex128` 张量（这一步才是主要内存开销）。

In [ ]:
# ===== 2. Pauli & tensor-product helpers (sparse) =====
sx = np.array([[0.0, 1.0], [1.0, 0.0]], dtype=complex)
sy = np.array([[0, -1j], [1j, 0]], dtype=complex)
sz = np.array([[1.0, 0.0], [0.0, -1.0]], dtype=complex)

_s = np.zeros((4, 2, 2), dtype=complex)  # 0:I, 1:X, 2:Y, 3:Z
_s[0, :, :] = np.eye(2, dtype=complex)
_s[1, :, :] = sx
_s[2, :, :] = sy
_s[3, :, :] = sz

def identity_n(L: int):
    """Return I^(tensor L) as a csr sparse matrix."""
    return csr_matrix(sp.eye(2**L, dtype=complex, format='csr'))

def sigma(i: int, j: int, L: int):
    """
    Build a single-qubit Pauli on an L-qubit register as a sparse matrix.

    i: 0->I, 1->X, 2->Y, 3->Z
    j: 1..L indicates the qubit index (1-based). If j==0, return I^(tensor L).
    """
    if j == 0:
        return identity_n(L)
    I2 = np.eye(2, dtype=complex)
    mat = _s[i] if j == 1 else I2
    for k in range(2, L+1):
        mat = np.kron(mat, _s[i] if k == j else I2)
    return csr_matrix(mat)

def sigmax(j, L): return sigma(1, j, L)
def sigmay(j, L): return sigma(2, j, L)
def sigmaz(j, L): return sigma(3, j, L)

print("[sanity] Hilbert dim =", 2**L)

## 3. 误差集合（Error sets）

脚本支持两种方式构造误差集合 $\mathcal{E}$：

### 3.1 旧风格（`ERROR_SET_MODE`）
- `orig`：$\{I\}\cup\{X_i,Y_i,Z_i\}_{i=1}^L$（单比特 Pauli）
- `E1`：在 `orig` 基础上加入若干二比特项（`XX, YY, XY, YX`，对所有 $i<j$）
- `E2`：`E1` 的闭包（对所有 $A,B\in E1$ 取乘积 $AB$）
- `E3(L,r)`：$\{I\}\cup\{X_i,Y_i\}\cup Z_{\le r}$，其中 $Z_{\le r}$ 是对至多 $r$ 个比特的 $Z$ 乘积（所有组合）

### 3.2 新风格（按“表格规格”）
通过 `build_error_set_from_table_spec(L,K,d,r)`：
- `d='2'`：等价于 `orig`
- `d='asym', r=None`：一个“混合非对称”集合  
  $\{I\}\cup\{X_i,Y_i,Z_i\}\cup\{XX,ZZ,XZ,ZX\}$
- `d='asym', r>=1`：等价于 `E3(L,r)`

这类“非对称”误差集常用于模拟 **相位翻转 (Z) 与位翻转 (X/Y)** 发生概率不同的信道结构。

In [ ]:
# ===== 3. Base Error Sets =====
def error_set_orig(L):
    E = [identity_n(L)]
    for i in range(1, L+1):
        E += [sigmax(i, L), sigmay(i, L), sigmaz(i, L)]
    return E

def error_set_E1(L):
    E = [identity_n(L)]
    for i in range(1, L+1):
        E += [sigmax(i, L), sigmay(i, L), sigmaz(i, L)]
    for i, j in combinations(range(1, L+1), 2):
        E.append(sigmax(i, L) @ sigmax(j, L))
        E.append(sigmay(i, L) @ sigmay(j, L))
        E.append(sigmax(i, L) @ sigmay(j, L))
        E.append(sigmay(i, L) @ sigmax(j, L))
    return E

def error_set_E2(L):
    E1 = error_set_E1(L)
    return [A @ B for A in E1 for B in E1]

def error_set_E3(L, r):
    E = [identity_n(L)]
    for i in range(1, L+1):
        E += [sigmax(i, L), sigmay(i, L)]
    for w in range(1, min(r, L)+1):
        for idxs in combinations(range(1, L+1), w):
            Zprod = identity_n(L)
            for j in idxs:
                Zprod = Zprod @ sigmaz(j, L)
            E.append(Zprod)
    return E

# ===== 3. Table-spec Error Sets =====
def error_set_asym_mixed(L):
    """{I} U {X_i,Y_i,Z_i} U {XX, ZZ, XZ, ZX for i<j}"""
    E = [identity_n(L)]
    for i in range(1, L+1):
        E += [sigmax(i, L), sigmay(i, L), sigmaz(i, L)]
    for i, j in combinations(range(1, L+1), 2):
        E.append(sigmax(i, L) @ sigmax(j, L))  # XX
        E.append(sigmaz(i, L) @ sigmaz(j, L))  # ZZ
        E.append(sigmax(i, L) @ sigmaz(j, L))  # XZ
        E.append(sigmaz(i, L) @ sigmax(j, L))  # ZX
    return E

def build_error_set_from_table_spec(L, K, d, r):
    """
    d='2'    -> {I} U {X_i,Y_i,Z_i}
    d='asym' & r is None -> {I} U {X_i,Y_i,Z_i} U {XX,ZZ,XZ,ZX}
    d='asym' & r>=1     -> {I} U {X_i,Y_i} U Z_{<=r}
    """
    d = str(d).lower()
    if d in ['2', 'two', 'sym', 'symmetric']:
        return error_set_orig(L)
    if d in ['asym', 'asymmetric']:
        if (r is None) or (str(r).strip() == '' or str(r).lower() == 'none'):
            return error_set_asym_mixed(L)
        r = int(r)
        if r < 1:
            raise ValueError("For d='asym', r must be >=1 or None.")
        return error_set_E3(L, r)
    raise ValueError("TABLE_D must be '2' or 'asym'.")

def build_error_set(mode, L, r=None):
    mode = mode.lower()
    if mode == 'orig': return error_set_orig(L)
    if mode == 'e1':   return error_set_E1(L)
    if mode == 'e2':   return error_set_E2(L)
    if mode == 'e3':
        if r is None: raise ValueError("E3 requires integer r.")
        return error_set_E3(L, r)
    raise ValueError(f"Unknown ERROR_SET_MODE: {mode}")

# quick check
if USE_TABLE_SPEC:
    Error_set = build_error_set_from_table_spec(L=L, K=TABLE_K, d=TABLE_D, r=TABLE_R)
    mode_desc = f"table_spec(d={TABLE_D}, r={TABLE_R}, n={L}, K={TABLE_K})"
else:
    Error_set = build_error_set(ERROR_SET_MODE, L, r=(TABLE_R if ERROR_SET_MODE.lower()=='e3' else None))
    mode_desc = ERROR_SET_MODE

print("[info] mode =", mode_desc)
print("[info] |Error_set| =", len(Error_set))

## 4. 从误差集合到 KL 约束算符列表（`op_list`）

我们真正拿来约束码空间的算符列表记为 $\{O_k\}_{k=1}^M$。

- **检测级（detect）**  
  直接用误差集合中的每个误差算符（可选丢掉 $I$）：
  \[
  O_k \in \mathcal{E}\setminus\{I\}.
  \]

- **纠错级（correct）**  
  KL 条件要求对所有 $E_a^\dagger E_b$ 都满足投影后为标量倍数。  
  对 Pauli 类（酉、Hermitian up to phase）算符，脚本用一个简单构造：
  \[
  O_{(a,b)} = E_a E_b,\quad a<b.
  \]
  这会形成更大的约束集合（通常比 detect 难很多）。

脚本中的 `feasible` 判据为：
\[
\text{offdiag}+\text{diag} < \texttt{constraint_threshold}.
\]
其中 offdiag/diag 由下一节的损失函数定义。

In [ ]:
# ===== 4. KL operator lists =====
def distance_3_error_set(error_set):
    ret = []
    n = len(error_set)
    for i in range(n):
        for j in range(i+1, n):
            ret.append(error_set[i] @ error_set[j])
    return ret

def is_identity(A, L):
    return (A - identity_n(L)).nnz == 0

def build_op_list_for_KL(Error_set, L, KL_LEVEL, drop_I_in_detect=True):
    level = KL_LEVEL.lower()
    if level == 'detect':
        return [A for A in Error_set if not (drop_I_in_detect and is_identity(A, L))]
    if level == 'correct':
        return distance_3_error_set(Error_set)
    raise ValueError("KL_LEVEL must be 'detect' or 'correct'.")

op_list_sparse = build_op_list_for_KL(Error_set, L, KL_LEVEL, drop_I_in_detect=DROP_IDENTITY_IN_DETECT)
print("[info] |op_list| =", len(op_list_sparse))

## 5. 可选：循环平移对称（cyclic symmetry, k=0）嵌入

这是脚本新增的一个重要功能：把搜索空间从全 Hilbert 空间 $\mathbb{C}^{2^L}$ 限制到 **循环不变子空间**（translation-invariant sector）。

### 5.1 直觉
定义循环平移算符 $T$：
\[
T\,|b_1 b_2 \dots b_L\rangle = |b_L b_1 \dots b_{L-1}\rangle.
\]
`k=0` 扇区对应 $T$ 的本征值为 $1$ 的子空间，即满足 $T|\psi\rangle=|\psi\rangle$。

### 5.2 构造方式（按轨道 orbit）
对任意计算基态 $|x\rangle$，考虑其在循环移位下的轨道：
\[
\mathcal{O}(x) = \{x, Tx, T^2x,\dots\}.
\]
在 `k=0` 子空间里，一个自然基向量是该轨道上的等幅叠加：
\[
|\Phi_{\mathcal{O}}\rangle = \frac{1}{\sqrt{|\mathcal{O}|}}\sum_{y\in \mathcal{O}} |y\rangle.
\]
把所有不同轨道的 $|\Phi_{\mathcal{O}}\rangle$ 作为列向量堆叠起来，就得到嵌入矩阵
\[
S\in\mathbb{C}^{2^L\times N_c},
\]
其中 $N_c$ 是轨道数量（也就是 `k=0` 子空间维度）。脚本最后还做了一次 QR 以数值稳健地正交化。

### 5.3 在优化里的作用
如果我们让 $U_{\rm eff}\in\mathbb{C}^{N_c\times K}$，则全空间码基为：
\[
U_{\rm full} = S\,U_{\rm eff}.
\]
并且每个算符在有效空间中的表示是：
\[
O_{\rm eff} = S^\dagger O S.
\]
这样所有优化都在更小的 $N_c$ 维空间里进行，往往快很多。

> 注意：开启 `USE_CYCLIC_SYM=True` 后，必须满足 `CODE_K <= N_c`，否则无解。

In [ ]:
# ===== 5. Cyclic-invariant (k=0) embedding =====
def _int_to_bits(i: int, L: int) -> tuple:
    return tuple((i >> (L-1-p)) & 1 for p in range(L))

def _bits_to_int(bits: tuple) -> int:
    v = 0
    for b in bits:
        v = (v << 1) | b
    return v

def _rotate_tuple(t: tuple, r: int = 1) -> tuple:
    n = len(t); r %= n
    return t[-r:] + t[:-r] if r else t

def _orbit_indices(i0: int, L: int) -> list:
    """Return the cyclic orbit of bitstring i0 (as integer indices)."""
    x = _int_to_bits(i0, L)
    seen = {}
    cur = x; r = 0
    while cur not in seen:
        seen[cur] = r
        r += 1
        cur = _rotate_tuple(cur, 1)
    orb = [w for w,_ in sorted(seen.items(), key=lambda kv: kv[1])]
    return [_bits_to_int(w) for w in orb]

def build_cyclic_k0_embedding(L: int) -> np.ndarray:
    """
    Return S in C^{2^L x N_c}, columns are normalized equal-amplitude orbit superpositions.
    Numerically, we QR-orthonormalize at the end for robustness.
    """
    N = 2**L
    used_reps = set()
    columns = []
    for i in range(N):
        orb = _orbit_indices(i, L)
        rep = min(orb)
        if rep in used_reps:
            continue
        used_reps.add(rep)
        s = len(orb)
        col = np.zeros(N, dtype=np.complex128)
        amp = 1.0/np.sqrt(s)
        for idx in orb:
            col[idx] = amp
        columns.append(col)
    S = np.stack(columns, axis=1)  # [N, N_c]
    Q, _ = np.linalg.qr(S)
    return Q.astype(np.complex128)

def describe_cyclic_embedding(L: int) -> str:
    S = build_cyclic_k0_embedding(L)
    return f"cyclic-k0 subspace (n={L}, dim={S.shape[1]})"

embedding = None
if USE_CYCLIC_SYM:
    assert CYCLIC_SECTOR.lower() == 'k0', "Only cyclic k=0 is implemented."
    S = build_cyclic_k0_embedding(L)   # [2^L, N_c]
    embedding = S
    print("[info] cyclic embedding:", S.shape, "|", describe_cyclic_embedding(L))
    if CODE_K > S.shape[1]:
        raise ValueError(f"CODE_K={CODE_K} > N_c={S.shape[1]} (cyclic subspace dim). Reduce K or disable symmetry.")
else:
    print("[info] cyclic embedding disabled.")

## 6. 优化变量：码空间等距嵌入 $U$（Stiefel 流形）

我们用一个矩阵 $U\in\mathbb{C}^{N\times K}$ 表示码空间的正交归一基（$N=2^L$），满足：
\[
U^\dagger U = I_K.
\]
这就是 **复 Stiefel 流形** $\mathrm{St}(N,K)$。

令 $P = U U^\dagger$ 为投影算符。对每个约束算符 $O_m$（来自上一节的 `op_list`），KL 条件希望：
\[
U^\dagger O_m U = \lambda_m I_K.
\]
脚本把这一目标分解成三个损失项：

1. **Off-diagonal penalty**  
   \[
   \mathcal{L}_{\text{off}}=\sum_m \|A_m-\mathrm{diag}(A_m)\|_F^2
   \]
   逼迫 $A_m$ 变成对角矩阵；

2. **Diagonal-equality penalty**  
   取对角线实部 $d_{m,i}=\mathrm{Re}(A_{m,ii})$，希望它们对 $i$ 不依赖（等于其均值）：
   \[
   \mathcal{L}_{\text{diag}}=\sum_m\sum_i (d_{m,i}-\bar d_m)^2.
   \]

3. **Lambda shaping**（扫描 $\lambda^{*2}$ 的关键）  
   令 $\lambda_m=\bar d_m$（每个算符对应一个标量），则
   \[
   \|\lambda\|^2 = \sum_m \lambda_m^2.
   \]
   当 `lambda_target = sqrt(lambda2)` 时，脚本用
   \[
   \mathcal{L}_\lambda = (\|\lambda\|^2 - \lambda^{*2})^2
   \]
   让解“贴近”指定的目标范数。

总损失：
\[
\mathcal{L} = \alpha(\mathcal{L}_{\text{off}}+\mathcal{L}_{\text{diag}})+\mathcal{L}_\lambda,
\]
其中 `alpha = model.penalty`。

> 重要：最终选择“可行解”时，只看 $\mathcal{L}_{\text{off}}+\mathcal{L}_{\text{diag}}$ 是否足够小。

In [ ]:
# ===== 6. Model (general K) with optional embedding =====
class DummyModel(torch.nn.Module):
    """
    Optimize U_eff on Stiefel(N_eff, K). If embedding S (N x N_eff) is provided:
      - U_full = S @ U_eff
      - We pre-project operators: O_eff = S^H O S (so optimization stays in N_eff).
    For each operator O_m, define A_m = U_eff^H O_eff U_eff, ideally lambda_m * I_K.
    """
    def __init__(self, op_list, penalty=1.0, embedding: np.ndarray = None):
        super().__init__()

        # sparse -> dense torch tensors: [M, N, N]
        op_dense = torch.stack([torch.tensor(op.toarray(), dtype=torch.complex128) for op in op_list])

        self.embedding = None
        if embedding is not None:
            S = torch.tensor(embedding, dtype=torch.complex128)    # [N, N_eff]
            Sdag = S.T.conj()                                      # [N_eff, N]
            # O_eff = S^H O S: [M, N_eff, N_eff]
            op_eff = torch.einsum('ab,mbc,cd->mad', Sdag, op_dense, S)
            self.op_list = op_eff
            self.embedding = S
            N_eff = S.shape[1]
        else:
            self.op_list = op_dense
            N_eff = op_dense.shape[-1]

        self.code_k = int(CODE_K)
        if self.code_k > N_eff:
            raise ValueError(f"CODE_K={self.code_k} > N_eff={N_eff}. Reduce K or disable symmetry.")

        self.manifold = numqi.manifold.Stiefel(N_eff, self.code_k, dtype=torch.complex128)
        self.lambda_target = None
        self.penalty = float(penalty)

    def set_lambda_target(self, x):
        """
        x can be:
          - None: disable lambda shaping
          - 'min'/'max': minimize or maximize ||lambda||^2
          - number (or array): target value(s)
        """
        if x is None:
            self.lambda_target = None
        elif isinstance(x, str):
            assert x in ['min', 'max']
            self.lambda_target = x
        else:
            t = torch.tensor(x, dtype=torch.float64).reshape(-1)
            self.lambda_target = t[0] if t.numel() == 1 else t

    def forward(self, return_info=False):
        U_eff = self.manifold()                           # [N_eff, K]
        # broadcasting matmul: (K,N_eff) @ (M,N_eff,N_eff) @ (N_eff,K) -> (M,K,K)
        A = U_eff.T.conj() @ self.op_list @ U_eff

        # Off-diagonal penalty
        diagA = torch.diagonal(A, dim1=1, dim2=2)         # [M, K]
        A_off = A - torch.diag_embed(diagA)
        loss_offdiag = (A_off.abs()**2).sum().real

        # Diagonal-equality penalty (use real part)
        diag_real = diagA.real                            # [M, K]
        diag_mean = diag_real.mean(dim=1, keepdim=True)   # [M, 1]
        loss_diag = ((diag_real - diag_mean)**2).sum()

        # Lambda shaping
        lambdas = diag_mean.squeeze(1)                    # [M]
        if self.lambda_target is None:
            loss_lambda = torch.tensor(0.0, dtype=torch.float64)
        elif isinstance(self.lambda_target, str):
            loss_lambda = torch.dot(lambdas, lambdas) if self.lambda_target == 'min' else -torch.dot(lambdas, lambdas)
        elif self.lambda_target.numel() == 1:
            loss_lambda = (torch.dot(lambdas, lambdas) - self.lambda_target**2)**2
        else:
            loss_lambda = torch.dot(lambdas - self.lambda_target, lambdas - self.lambda_target)

        total = self.penalty*(loss_offdiag + loss_diag) + loss_lambda

        if return_info:
            if self.embedding is not None:
                U_full = self.embedding @ U_eff
            else:
                U_full = U_eff
            total = total, dict(
                loss=(total, loss_offdiag, loss_diag, loss_lambda),
                lambda_ab_ij=A,       # A in effective space
                code=U_full,          # basis in full space
                code_eff=U_eff        # basis in effective space
            )
        return total

# Instantiate model
model = DummyModel(op_list_sparse, embedding=embedding)
model.penalty = 10.0
print("[info] model built; penalty =", model.penalty)

## 7. 主流程：扫描 $\lambda^{*2}$，记录“可行最优”并保存图表

脚本做的事情可以概括为：

1. 对每个目标点 $\lambda^{*2}\in$ `lambda2_list`：
   - 设定 `lambda_target = sqrt(lambda2)`；
   - 用 `numqi.optimize.minimize` 在 Stiefel 流形上优化；
   - 收集每次迭代的参数 `optim_x`，并**重新回放**计算四个损失 + 实现的 $\|\lambda\|^2$；
2. 以约束阈值判断可行：
   - `feasible(row) := offdiag + diag < constraint_threshold`
3. 对每个 $\lambda^{*2}$ 选出 “lambda_loss 最小的可行解” 作为该点代表；
4. 输出：
   - 控制台打印每个目标点的最优可行值（含 `achieved_lambda2`）
   - 画 `lambda_loss` vs `lambda2`
   - 保存 `png` 图与 `csv` 摘要

下面的代码基本等价于你脚本里的 `main()`，但我们把它封装成函数，方便在 Notebook 里重复调用或改参数。

In [ ]:
# ===== 7. Main scan function =====
def run_lambda2_scan(
    model: DummyModel,
    lambda2_list: np.ndarray,
    optim_kwargs: dict,
    constraint_threshold: float,
    out_prefix: str = None,
):
    """
    Run scan over target lambda2 values.
    Return:
      - fval_optim: [T,5] = [total, offdiag, diag, lambda_loss, achieved_lambda2] for best feasible point
      - loss_histories: list of histories; each history is a list of rows like above for each iterate
    """
    loss_histories = []
    codes = []

    for lambda2 in lambda2_list:
        model.set_lambda_target(np.sqrt(float(lambda2)))
        callback = numqi.optimize.MinimizeCallback(print_freq=500)
        _ = numqi.optimize.minimize(model, callback=callback, **optim_kwargs)

        # replay every iterate to compute achieved ||lambda||^2 precisely
        with torch.no_grad():
            hist = []
            for h in callback.history_state:
                numqi.optimize.set_model_flat_parameter(model, h['optim_x'])
                loss, info = model(return_info=True)

                A = info['lambda_ab_ij']                          # [M,K,K]
                diagA = torch.diagonal(A, dim1=1, dim2=2).real    # [M,K]
                lam_vec = diagA.mean(dim=1)                       # [M]
                lam2_val = float(torch.dot(lam_vec, lam_vec))     # achieved ||lambda||^2

                four_losses = [x.item() for x in info['loss']]    # [total, off, diag, lambda_loss]
                hist.append(four_losses + [lam2_val])             # + achieved_lambda2
            print('[scan] target lambda2 =', float(lambda2), '| iterates =', len(hist))
            loss_histories.append(hist)

        # save final code basis for this target
        codes.append(model(return_info=True)[1]['code'])

    # feasibility and selection
    def feasible(row):
        return (row[1] + row[2]) < constraint_threshold

    def best(history):
        feas = [x for x in history if feasible(x)]
        return [np.nan]*5 if len(feas)==0 else min(feas, key=lambda x: x[3])  # minimize lambda_loss

    fval_optim = np.array([best(h) for h in loss_histories])  # [T, 5]

    # report
    print("\n[report] Best feasible per target lambda2")
    print(" idx | target_lambda2 | feas? | loss_total |  offdiag |    diag  | lambda_loss | achieved_lambda2")
    for i, (target_l2, hist) in enumerate(zip(lambda2_list, loss_histories)):
        feas_hist = [x for x in hist if feasible(x)]
        if not feas_hist:
            print(f"{i:4d} | {target_l2:14.6g} |  no  |     nan    |    nan  |    nan  |    nan     |      nan")
            continue
        row = min(feas_hist, key=lambda x: x[3])
        print(f"{i:4d} | {target_l2:14.6g} | yes  | {row[0]:10.3e} | {row[1]:8.1e} | {row[2]:8.1e} | {row[3]:11.3e} | {row[4]:14.6g}")

    # plot: lambda_loss curve
    fig, ax = plt.subplots()
    ax.plot(lambda2_list, fval_optim[:, 3], label='lambda_loss (best feasible)')
    ax.set_xlabel(r'$\lambda^{*2}$')
    ax.set_ylabel('lambda loss')
    ax.set_yscale('log')
    ax.grid(True, which='both', linestyle='--', alpha=0.4)
    ax.yaxis.set_major_locator(LogLocator(base=10))
    ax.yaxis.set_minor_formatter(ScalarFormatter())

    max_constraint = np.nan_to_num(fval_optim[:,1] + fval_optim[:,2], nan=0).max()
    sym_tag = " | cyclic k=0" if USE_CYCLIC_SYM else ""
    mode_desc = f"{'table_spec' if USE_TABLE_SPEC else ERROR_SET_MODE}(d={TABLE_D}, r={TABLE_R}, n={L}, K={TABLE_K})"
    ax.set_title(f"Cyclic Codes vs lambda: constraint <= {max_constraint:.3g} | mode={mode_desc} | KL={KL_LEVEL} | K={CODE_K}{sym_tag}")
    ax.legend(loc='best', fontsize=9)
    fig.tight_layout()

    # optional save
    if out_prefix is not None:
        out_png = f"{out_prefix}.png"
        fig.savefig(out_png, dpi=200)
        print(f'[plot] saved -> {out_png}')

        out_csv = f"{out_prefix}.csv"
        data_csv = np.column_stack([
            lambda2_list,
            fval_optim[:,0],
            fval_optim[:,1],
            fval_optim[:,2],
            fval_optim[:,3],
            fval_optim[:,4],
        ])
        header = "target_lambda2,total,offdiag,diag,lambda_loss,achieved_lambda2"
        np.savetxt(out_csv, data_csv, delimiter=",", header=header, comments="")
        print(f"[save] summary -> {out_csv}")

    return fval_optim, loss_histories, codes

# output name base (match original)
sym_suffix = "_cyclic" if USE_CYCLIC_SYM else ""
out_prefix = f"n{L}_K{CODE_K}_{TABLE_D}_r{TABLE_R}_KL-{KL_LEVEL}{sym_suffix}"

print("[info] out_prefix =", out_prefix)

## 8. 运行扫描（可能耗时）

- 如果你打开了 `SMOKE_TEST=True`，这里会很快跑完并生成 `png/csv`。
- 如果你使用原始默认 `lambda2_list`（100 个点）且 `num_repeat=50`，可能需要较长时间。

运行后当前目录会生成：
- `{out_prefix}.png`
- `{out_prefix}.csv`

你也可以把 `out_prefix` 改成带路径的字符串（例如 `"results/myrun"`），把输出放到某个文件夹里。

In [ ]:
# ===== 8. Run the scan =====
# 注意：这一步可能很慢。建议先 SMOKE_TEST=True 试跑。
fval_optim, loss_histories, codes = run_lambda2_scan(
    model=model,
    lambda2_list=lambda2_list,
    optim_kwargs=optim_kwargs,
    constraint_threshold=constraint_threshold,
    out_prefix=out_prefix,
)

# fval_optim: [T,5] -> [total, offdiag, diag, lambda_loss, achieved_lambda2]
print("[done] fval_optim shape =", fval_optim.shape)

## 9. （可选）后验检查：抽样查看 $U^\dagger O U$

为了更直观地验证 KL 结构，我们可以随机抽几个算符 $O_k$，打印矩阵
\[
A_k = U^\dagger O_k U
\]
的前几行列，观察：
- 是否近似对角；
- 对角线实部是否接近常数；
- off-diagonal Frobenius 范数是否很小。

下面函数与原脚本一致，适配任意 $K$，也兼容开启 embedding 的情况。

In [ ]:
# ===== 9. Optional QC =====
def qc_sample(model: DummyModel, idx_list=None, max_print=3):
    """Sample some operators and print U^H O_k U for quick inspection."""
    with torch.no_grad():
        loss, info = model(return_info=True)
        A = info['lambda_ab_ij']    # [M,K,K] in effective space
        M, K = A.shape[0], A.shape[-1]
        if idx_list is None:
            idx_list = np.random.choice(M, size=min(max_print, M), replace=False)
        for k in idx_list:
            Mk = A[k].cpu().numpy()
            print(f"[QC] op idx={k}, matrix shape={Mk.shape}, K={K}")
            sl = slice(0, min(K, 4))
            print(np.round(Mk[sl, sl], 6))
            off = Mk - np.diag(np.diag(Mk))
            print("-- offdiag Fro norm^2:", float((abs(off)**2).sum()))
            d = np.real(np.diag(Mk))
            print("-- diag real:", np.round(d, 6), " (std:", float(d.std()), ")")

# Example usage (after running the scan, model holds the last solution):
# qc_sample(model)

## 10. 常见改法与排错建议

1. **想更严格的纠错级码**：把 `KL_LEVEL='correct'`。  
   注意：`op_list` 会暴涨（$\binom{|\mathcal{E}|}{2}$ 级别），计算会变慢/内存更大。

2. **想加入循环对称**：把 `USE_CYCLIC_SYM=True`。  
   - 会先构造 `S`，并把每个算符投影到有效空间 `O_eff = S^H O S`；
   - 如果报错 `CODE_K > N_c`，说明你想要的码维度超过了循环不变子空间的维度，需要减小 `K` 或关掉对称性。

3. **速度/稳定性**：
   - 第一次先 `SMOKE_TEST=True`；
   - 把 `optim_kwargs['num_repeat']` 从 50 降到 5~10；
   - `constraint_threshold` 放宽到 `1e-9` 或 `1e-8` 先找趋势，再收紧。

4. **内存爆炸**：
   - 主要发生在 `op_dense = torch.stack([... op.toarray() ...])`  
     这里会把 `M` 个 $2^L\times 2^L$ 矩阵一次性堆起来；
   - 如果要做更大 $L$，通常要改成：
     - 只保留必要的算符；
     - 或利用张量积结构/稀疏结构/块结构在乘法里避免显式稠密化。

---

到这里，Notebook 已把原脚本的“讲义版”整理完毕：你可以直接改参数重复实验，也可以把某些 cell 抽出来做更系统的数值研究（例如对比不同误差集、不同 KL_LEVEL、不同对称扇区等）。